In [2]:
import pandas as pd
from functions import assembleDf
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time


In [106]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

In [67]:
epochG(matches[1:10],{},matches.iloc[10]['tourney_date'])

({113953: <pyglicko2.glicko2.Player at 0x7fc8a00089d0>,
  114146: <pyglicko2.glicko2.Player at 0x7fc8a0008970>,
  113987: <pyglicko2.glicko2.Player at 0x7fc8a0008850>,
  114147: <pyglicko2.glicko2.Player at 0x7fc8a0008880>,
  114149: <pyglicko2.glicko2.Player at 0x7fc8a00084c0>,
  114154: <pyglicko2.glicko2.Player at 0x7fc8a00085e0>,
  113963: <pyglicko2.glicko2.Player at 0x7fc8a0008430>,
  114158: <pyglicko2.glicko2.Player at 0x7fc8a0008100>,
  113999: <pyglicko2.glicko2.Player at 0x7fc8a0008250>,
  113976: <pyglicko2.glicko2.Player at 0x7fc8a0008730>,
  113982: <pyglicko2.glicko2.Player at 0x7fc8a00087f0>},
 {(113987, Timestamp('1877-07-09 00:00:00')): 1799.6258356900184,
  (113963, Timestamp('1877-07-09 00:00:00')): 1599.8752635134226,
  (113999, Timestamp('1877-07-09 00:00:00')): 1599.8752635134226,
  (114158, Timestamp('1877-07-09 00:00:00')): 1500.0,
  (114149, Timestamp('1877-07-09 00:00:00')): 1662.3108949741131,
  (113953, Timestamp('1877-07-09 00:00:00')): 1337.6891050258869,

In [78]:
%%time
start = time.time()
pd,rh = epochsG(matches[256_000:360_000],180)
compute_time = time.time()-start
print(compute_time)


9.954494953155518
CPU times: user 9.92 s, sys: 41.5 ms, total: 9.96 s
Wall time: 9.95 s
